# Maximum Likelihood Estimation of ARMA-GARCH

One possible approach of fitting an ARMA-GARCH model is to perform a maximum likelihood estimation (MLE) for the conditional mean (ARMA), then an MLE of the conditional variance (GARCH). However, joint estimation is preferred. In the first stage of ARMA estimation, there is an implicit assumption of conditional homoskedasticity. It is contradicted in the second stage when you explicitly model conditional heteroskedasticity using GARCH.

## An Example: GARCH(1,1) with Normal Distribution

Recall a GARCH(1,1) model is defined as:

\begin{equation}
    \sigma_t^2 = \alpha_0 + \alpha_1 \epsilon_{t-1}^2 + \beta_1 \sigma_{t-1}^2,
\end{equation}

and the *log-likelihood* function for a normally distributed random variable is:

\begin{equation}
    L = - \frac{1}{2} \sum_{t=1}^T \left( \ln \sigma_t^2 + \left(\frac{\epsilon_{t}}{\sigma_t} \right)^2 \right),
\end{equation}

In [23]:
import numpy as np

def garch(alpha0, alpha1, beta1, epsilon):
    T = len(epsilon)
    sigma_2 = np.zeros(T)
    
    for t in range(T):
        if t == 0:
            sigma_2[t] = alpha0 / (1 - alpha1 - beta1) # initialize as unconditional variance
        else:
            sigma_2[t] = alpha0 + alpha1*epsilon[t-1]**2 + beta1*sigma_2[t-1]
            
    return sigma_2
    
def garch_neg_loglike(params, epsilon):
    T = len(epsilon)
    alpha0 = params[0]
    alpha1 = params[1]
    beta1 = params[2]
    sigma_2 = garch(alpha0, alpha1, beta1, epsilon)
    NegLogL = -0.5 * np.sum(-np.log(sigma_2) - epsilon**2/sigma_2)  # negative sign for minimization
    return NegLogL

In [24]:
# load data

import pandas as pd
from scipy.optimize import minimize

data = pd.read_csv('data/top10_logreturns.csv', index_col=0, parse_dates=True)['D05.SI'] * 100  # scaled for ease of optimization
bounds = tuple((0.0001, None) for i in range(3))
params_initial = (0.1, 0.05, 0.92)
cons = (
    {'type': 'ineq', 'func': lambda x: np.array(x)},
    {'type': 'ineq', 'func': lambda x: 1-x[1]-x[2]+0.00000000000001}
)  

res = minimize(garch_neg_loglike, params_initial, args=(data), bounds=bounds, options={'disp': True})
res

      fun: 1475.3801857119552
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00350155, -0.00422915, -0.00513865])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 76
      nit: 16
     njev: 19
   status: 0
  success: True
        x: array([0.06084818, 0.13416952, 0.82254072])

## Another Example: ARMA(1,1) with Normal Distrbitution

We utilise the same log-likelihood function defined above, but this time we use an ARMA(1,1) model,

\begin{equation}
    \epsilon_t = r_t - \phi_0 - \phi_1 r_{t-1} - \theta_1 \epsilon_{t-1},
\end{equation}

which has been rearranged to solve for $\epsilon_t$.

In [25]:
def arma(phi0, phi1, theta1, r):
    T = len(r)
    epsilon = np.zeros(T)
    
    for t in range(T):
        if t == 0:
            epsilon[t] = r[t] - np.mean(r)
        else:
            epsilon[t] = r[t] - phi0 - phi1*r[t-1] - theta1*epsilon[t-1]
    
    return epsilon

def arma_neg_loglike(params, r):
    T = len(r)
    phi0 = params[0]
    phi1 = params[1]
    theta1 = params[2]
    epsilon = arma(phi0, phi1, theta1, r)
    NegLogL = -0.5 * np.sum(-np.log(r.var()) - epsilon**2/r.var())
    return NegLogL

bounds = tuple((0.0001, None) for i in range(3))
params_initial = (0.1, 0.05, 0.92)
cons = ({'type': 'ineq', 'func': lambda x: np.array(x)})  

res = minimize(arma_neg_loglike, params_initial, args=(data), bounds=bounds, options={'disp': True})
res

## Fitting ARMA-GARCH with MLE

Based on some preliminary research, many sources suggest estimating the *ARMA* process first, followed by modelling the innovations with GARCH. However, this will most likely lead to inconsistent parameter estimates. In fitting an ARMA model, there is an assumption made about the *conditional variance* - it is constant. This is clearly not the case when the process is assumed to follow that of GARCH. This is especially an issue when it comes to order determination for the ARMA model - the ACF and PACF confidence bounds will be invalid given the GARCH-type residuals. Therefore, parameter determination via MLE must be performed for both ARMA and GARCH *simultaneously*. This simply involves substituting the *conditional mean* component from ARMA and the *conditional variance* component from GARCH into the log-likelihood equation and minimizing with `scipy`.

In [27]:
def armagarch_negloglike(params, r):
    T = len(r)
    phi0 = params[0]
    phi1 = params[1]
    theta1 = params[2]
    alpha0 = params[3]
    alpha1 = params[4]
    beta1 = params[5]
    
    epsilon = arma(phi0, phi1, theta1, r)
    sigma_2 = garch(alpha0, alpha1, beta1, epsilon)
    
    NegLogL = -0.5 * np.sum(-np.log(sigma_2) - epsilon**2/sigma_2)
    
    return NegLogL

In [34]:
bounds = tuple((0.0001, None) for i in range(6))
params_initial = tuple(0.0001 for _ in range(6))
cons = ({'type': 'ineq', 'func': lambda x: np.array(x)},
        {'type': 'ineq', 'func': lambda x: 1-x[-1]-x[-2]+0.00000000000001})  

res = minimize(armagarch_negloglike, params_initial, args=(data), bounds=bounds, options={'disp': True})
res

<ipython-input-27-1a29fe4b8cf4>:13: RuntimeWarning: invalid value encountered in log
  NegLogL = -0.5 * np.sum(-np.log(sigma_2) - epsilon**2/sigma_2)


      fun: 1468.1974203477923
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00613909,  0.39729002, -0.0017053 ,  0.01248281, -0.00616183,
        0.02321485])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 427
      nit: 42
     njev: 61
   status: 0
  success: True
        x: array([4.94361056e-02, 1.00000000e-04, 5.64613715e-02, 5.45187055e-02,
       1.27127949e-01, 8.33942576e-01])

In [40]:
# AIC calculate as 2k - 2LogL or 
# AIC = 2k + 2NegLogL

AIC = 2 * len(params_initial) + 2* res.fun
AIC

2948.3948406955847

` model = Model(mean='ARMA', var='GARCH', data=data)